In [3]:
import numpy as np
import pandas as pd

bookDF=pd.read_csv('books.csv', error_bad_lines=False) # DATASET OF BOOKS
bookDF=bookDF.drop(['small_image_url','title','best_book_id','isbn','isbn13'], axis=1)
ratingsDF = pd.read_csv('ratings.csv', error_bad_lines=False) # DATASET OF RATINGS

In [6]:
bookDF.head()

,id,book_id,work_id,books_count,authors,original_publication_year,original_title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
0,1,2767052,2792775,272,Suzanne Collins,2008.0,The Hunger Games,eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317
1,2,3,4640799,491,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543
2,3,41865,3212258,226,Stephenie Meyer,2005.0,Twilight,en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439
3,4,2657,3275794,487,Harper Lee,1960.0,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267
4,5,4671,245494,1356,F. Scott Fitzgerald,1925.0,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718


In [9]:
ratingsDF.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [10]:
listOfDictonaries=[] #EACH DICT IS A BOOK, KEY: USER_ID VALUE: RATING
indexMap = {}
reverseIndexMap = {}
ptr=0;
testdf = ratingsDF
testdf=testdf[['user_id','rating']].groupby(testdf['book_id'])
for groupKey in testdf.groups.keys():
    tempDict={}

    groupDF = testdf.get_group(groupKey)
    for i in range(0,len(groupDF)):
        tempDict[groupDF.iloc[i,0]]=groupDF.iloc[i,1]
    indexMap[ptr]=groupKey
    reverseIndexMap[groupKey] = ptr
    ptr=ptr+1
    listOfDictonaries.append(tempDict)

In [18]:
listOfDictonaries[0] # [{key: value}, {}, {}]

{314: 5,
 439: 3,
 588: 5,
 1169: 4,
 1185: 4,
 2077: 4,
 2487: 4,
 2900: 5,
 3662: 4,
 3922: 5,
 5379: 5,
 5461: 3,
 5885: 5,
 6630: 5,
 7563: 3,
 9246: 1,
 10140: 4,
 10146: 5,
 10246: 4,
 10335: 4,
 10610: 5,
 10944: 5,
 11854: 4,
 11927: 4,
 12471: 5,
 13282: 5,
 13544: 5,
 15494: 5,
 16377: 4,
 16913: 5,
 17434: 5,
 17663: 5,
 17984: 5,
 18031: 5,
 18313: 5,
 18361: 4,
 20076: 3,
 20467: 3,
 20848: 5,
 21228: 5,
 21487: 4,
 21713: 5,
 22602: 4,
 23576: 4,
 23612: 4,
 24326: 5,
 24389: 4,
 24499: 5,
 24834: 4,
 24845: 5,
 25164: 4,
 25182: 5,
 25214: 4,
 26145: 4,
 26629: 4,
 26661: 4,
 28158: 4,
 28767: 5,
 29123: 3,
 29703: 5,
 30681: 5,
 31001: 4,
 32055: 4,
 32305: 5,
 32592: 4,
 32635: 5,
 32748: 5,
 32923: 5,
 33065: 4,
 33697: 4,
 33716: 5,
 33872: 5,
 33890: 3,
 37284: 5,
 37834: 5,
 38080: 4,
 38082: 5,
 38475: 4,
 39423: 3,
 41074: 3,
 42404: 5,
 43985: 4,
 44243: 2,
 44397: 5,
 45269: 4,
 45493: 5,
 46977: 4,
 47476: 4,
 47746: 5,
 47800: 5,
 48482: 3,
 49298: 4,
 50104:

In [20]:
from sklearn.feature_extraction import DictVectorizer
dictVectorizer = DictVectorizer() # CREATE VECTORS, EACH POINT IS A BOOK, RATING IS IT'S MAGNITUDE
vector = dictVectorizer.fit_transform(listOfDictonaries)

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
pairwiseSimilarity = cosine_similarity(vector) # SIMILARITY BETWEEN BOOKS

In [26]:
def printBookDetails(bookID):
    print("Title:", bookDF[bookDF['id']==bookID]['original_title'].values[0])
    print("Author:",bookDF[bookDF['id']==bookID]['authors'].values[0])
    print("Book-ID:",bookID)
    print("\n ==================================================================================== \n")


def getTopRecommandations(bookID):
    row = reverseIndexMap[bookID]
    print("------INPUT BOOK--------")
    printBookDetails(bookID)
    print("-------RECOMMENDATIONS----------")
    similarBookIDs = [printBookDetails(indexMap[i]) for i in np.argsort(pairwiseSimilarity[row])[-7:-2][::-1]]

In [28]:
getTopRecommandations(500) # INPUT BOOK ID BETWEEN 1 TO 10,000

------INPUT BOOK--------
Title: The Princess Diaries
Author: Meg Cabot
Book-ID: 500


-------RECOMMENDATIONS----------
Title: Princess in the Spotlight
Author: Meg Cabot
Book-ID: 3452


Title: Ella Enchanted
Author: Gail Carson Levine
Book-ID: 263


Title: A Great and Terrible Beauty
Author: Libba Bray
Book-ID: 586


Title: All-American Girl
Author: Meg Cabot
Book-ID: 2278


Title: Princess in Waiting (The Princess Diaries, #4)
Author: Meg Cabot
Book-ID: 4345


